Датасет - https://www.kaggle.com/datasets/rkiattisak/student-performance-in-mathematics

Предметная область - успеваемость старшеклассников

Источник данных - генератор датасетов

Тип данных - имитационные

Основная задача анализа - выявление зависимостей между различными атрибутами и успеваемостью

Атрибуты датасета:

• Gender: Пол студента (мужчина/женщина)

• Race: Расовое или этническое происхождение студента

• Education: Самый высокий уровень образования, достигнутый родителями или опекунами учащегося

• Lunch: Получает ли студент бесплатный обед или обед по сниженной цене

• Preparation: Прошел ли студент курс подготовки к тестированию

• Math score: Оценка учащегося по стандартизированному тесту по математике

• Reading score: Оценка учащегося по стандартизированному тесту на чтение

• Writing score: Оценка студента по стандартизированному письменному тесту

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

names = ['Gender', 'Race', 'Education', 'Lunch', 'Preparation', 'Math score', 'Reading score', 'Writing score']

csv = pd.read_csv('exams.csv', sep=',')
csv.columns = names
df = pd.DataFrame(csv)

# Конвертация параметров в числовые
df['Gender'] = df['Gender'].replace({'male': 0, 'female': 1})
df['Race'] = df['Race'].replace({'group A': 0, 'group B': 1, 'group C': 2, 'group D': 3, 'group E': 4})
df['Education'] = df['Education'].replace({'high school': 0, 'some high school': 0, 'some college': 1,
                                           "master's degree": 2,"bachelor's degree": 3, "associate's degree": 4})
df['Lunch'] = df['Lunch'].replace({'standard': 0, 'free/reduced': 1})
df['Preparation'] = df['Preparation'].replace({'none': 0, 'completed': 1})

df['Average score'] = (df['Math score'] + df['Reading score'] + df['Writing score']) / 3

Во второй работе был проведен анализ датасета, выбросы и пропущенные значения отсутствуют

По аналогии со второй работой добавим атрибут Average score, который выступит в роли целевого класса

In [21]:
# Разделим средний балл по трем экзаменам на удовлетворительные и неудовлетворительные на основании среднего значения атрибута
df['Average score'] = (df['Average score'] >= df['Average score'].mean()).astype(int)
df.sample(5)

,Gender,Race,Education,Lunch,Preparation,Math score,Reading score,Writing score,Average score
768,1,2,0,0,0,70,75,78,1
334,0,3,1,1,0,59,45,44,0
452,1,2,0,1,0,38,45,44,0
827,0,3,1,0,0,79,56,68,0
272,1,4,0,1,0,74,88,75,1


In [22]:
y = df["Average score"]
df.drop(["Average score"], axis=1, inplace=True)

In [23]:
from sklearn.model_selection import StratifiedKFold, train_test_split

# Выделим 70% датасета под обучение и 30% оставим для оценки качества модели
X_train, X_holdout, y_train, y_holdout = train_test_split(df.values, y, test_size=0.3, random_state=17)

In [24]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# Обучим дерево решений и kNN
tree = DecisionTreeClassifier(max_depth=5, random_state=17)
knn = KNeighborsClassifier(n_neighbors=10)

tree.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=10)

In [25]:
from sklearn.model_selection import GridSearchCV, cross_val_score

# Найдем лучшие параметры для дерева решений
tree_params = {"max_depth": range(1, 11), "max_features": range(4, 19)}
tree_grid = GridSearchCV(tree, tree_params, cv=5, n_jobs=-1, verbose=True)
tree_grid.fit(X_train, y_train)
print(f"Дерево решений\nЛучший набор параметров: {tree_grid.best_params_}\nДоля правильных ответов: {tree_grid.best_score_}")

Fitting 5 folds for each of 150 candidates, totalling 750 fits
Дерево решений
Лучший набор параметров: {'max_depth': 4, 'max_features': 7}
Доля правильных ответов: 0.9785714285714286


In [26]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Найдем лучшие параметры для kNN
knn_pipe = Pipeline([("scaler", StandardScaler()), ("knn", KNeighborsClassifier(n_jobs=-1))])
knn_params = {"knn__n_neighbors": range(1, 10)}
knn_grid = GridSearchCV(knn_pipe, knn_params, cv=5, n_jobs=-1, verbose=True)
knn_grid.fit(X_train, y_train)
print(f"kNN\nЛучший набор параметров: {knn_grid.best_params_}\nДоля правильных ответов: {knn_grid.best_score_}")

Fitting 5 folds for each of 9 candidates, totalling 45 fits
kNN
Лучший набор параметров: {'knn__n_neighbors': 7}
Доля правильных ответов: 0.9271428571428573


In [27]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

print("Дерево решений")
print(f'Accuracy: {accuracy_score(y_holdout, tree_grid.predict(X_holdout))}')
print(f'Presicion: {precision_score(y_holdout, tree_grid.predict(X_holdout))}')
print(f'Recall: {recall_score(y_holdout, tree_grid.predict(X_holdout))}')
print(f'F-measure: {f1_score(y_holdout, tree_grid.predict(X_holdout))}')
print(f'ROC: {roc_auc_score(y_holdout, tree_grid.predict(X_holdout))}')

Дерево решений
Accuracy: 0.97
Presicion: 0.968944099378882
Recall: 0.975
F-measure: 0.9719626168224299
ROC: 0.9696428571428571


In [28]:
print("kNN")
print(f'Accuracy: {accuracy_score(y_holdout, knn_grid.predict(X_holdout))}')
print(f'Presicion: {precision_score(y_holdout, knn_grid.predict(X_holdout))}')
print(f'Recall: {recall_score(y_holdout, knn_grid.predict(X_holdout))}')
print(f'F-measure: {f1_score(y_holdout, knn_grid.predict(X_holdout))}')
print(f'ROC: {roc_auc_score(y_holdout, knn_grid.predict(X_holdout))}')

kNN
Accuracy: 0.94
Presicion: 0.9551282051282052
Recall: 0.93125
F-measure: 0.9430379746835443
ROC: 0.9406249999999999


На данном датасете по всем параметрам лучше себя показало дерево решений


Самая высокая разница в полноте (0.04)

Меньше разница в доле правильных ответов (0.03), F-мере (0.03) и ROC (0.03)

Самая маленькая разница в точности (0.01)